In [19]:
import random
import time
%run pravljenjeGrafova.ipynb

#Stvara nasumicnu pocetnu populaciju
def pocetnaPopulacija(size, brojCvoreva):
    l1 = [[random.choice([0, 1]) for _ in range(brojCvoreva)] for _ in range(0,size)]
    l2 = [[]]
    for l in l1:
        l2.append([str(index+1) for index, value in enumerate(l) if value == 1])

    filtriranaLista = [unutrasnja for unutrasnja in l2 if unutrasnja]  #Izbaci prazne liste
    return filtriranaLista

# Operator selekcije (ocjenjuje prema broju cvorova, ukoliko nije slabo povezan dom skup vraca 1000)
def selekcija(jedinka, graf):

    if jelSlaboPovezan(jedinka.copy(), graf):
        return len(jedinka)
    else:
        return 1000  # Velika ocjena za neslabo povezane dom skupove

{'1': ['6', '9', '10', '13', '14', '15', '18', '19', '20', '21', '24', '25', '26', '27', '28'], '2': ['5', '8', '10', '12', '14', '15', '17', '19', '20', '21', '23', '25', '26', '27', '28'], '3': ['4', '7', '10', '11', '14', '15', '16', '19', '20', '21', '22', '25', '26', '27', '28'], '4': ['3', '8', '9', '12', '13', '15', '17', '18', '20', '21', '23', '24', '26', '27', '28'], '5': ['2', '7', '9', '11', '13', '15', '16', '18', '20', '21', '22', '24', '26', '27', '28'], '6': ['1', '7', '8', '11', '12', '15', '16', '17', '20', '21', '22', '23', '26', '27', '28'], '7': ['3', '5', '6', '12', '13', '14', '17', '18', '19', '21', '23', '24', '25', '27', '28'], '8': ['2', '4', '6', '11', '13', '14', '16', '18', '19', '21', '22', '24', '25', '27', '28'], '9': ['1', '4', '5', '11', '12', '14', '16', '17', '19', '21', '22', '23', '25', '27', '28'], '10': ['1', '2', '3', '11', '12', '13', '16', '17', '18', '21', '22', '23', '24', '27', '28'], '11': ['3', '5', '6', '8', '9', '10', '17', '18', '19',

In [20]:
# Funkcija za provjeru slabog povezivanja
def nadjiSusjede (graf,cvor):
    return graf[cvor].copy()

def jelSlaboPovezan(dominirajuci,graf):

    if len(dominirajuci)==0:
        return False

    trcvor = dominirajuci[0]
    del dominirajuci[0]
    
    zaObici = []
    oznaceni = set()
    
    oznaceni.add(trcvor)
    zaObici = nadjiSusjede(graf,trcvor)

    for l in zaObici:
            if l not in oznaceni:
                oznaceni.add(l)     

    while not len(graf)==len(oznaceni):

        if (len(zaObici)==0):
            break
        
        trcvor = zaObici[0]
        zaObici.remove(trcvor)

        #ako jeste dominirajuci
        if trcvor in dominirajuci:

            dominirajuci.remove(trcvor)          
            oznaceni.add(trcvor)
            lista1 = nadjiSusjede(graf,trcvor)

            for l in lista1:
                if  l not in oznaceni:
                    zaObici.append(l)
                oznaceni.add(l)     
        #ako nije dom onda uzimamo njegove susjede koji su dom i dodajemo ih za pretragu
        else:
            lista1 = nadjiSusjede(graf,trcvor)
            for l in lista1:
                if l in dominirajuci:
                    zaObici.append(l)

    #Ako smo izbrisali sve dominirajuce i ako su oznaceni svi cvorovi iz grafa, znaci da jeste slabo povezan DS
    if(len(dominirajuci)==0):
        if (len(oznaceni)==len(graf)):
            return True
        else:
            return False
    return False

In [21]:
#Operator ukrstanja
def ukrstanje(parent1, parent2):
    ukrstanje_point = random.randint(0, len(parent1))
    child = parent1[:ukrstanje_point] + parent2[ukrstanje_point:]
    child = list(set(child)) #Uklanjamo duplikate
    return child

#Operator mutacije
def mutacija(jedinka,brC):
    for i in range(brC):
        if random.random() < vjvMutacije:         
            if (i+1) in jedinka:
                jedinka.remove(i+1)
    return jedinka

In [22]:
# Genetski algoritam i parametri

brojPopulacije = 250
vjvMutacije = 0.1

def genetskiAlgoritam(graf,vrijemeTrajanja):

    brojCvoreva = len(graf)
    populacija = pocetnaPopulacija(brojPopulacije, brojCvoreva)
    print("Pocetna populacija",populacija)
    startTime = time.time()

    while True:
        
        populacija.sort(key=lambda jedinka: selekcija(jedinka, graf))

        elitneJedinke = populacija[:2]
        ostaleJedinke = random.sample(populacija[2:], brojPopulacije - 2)
        parents = elitneJedinke + ostaleJedinke

        novaGen = []
        for e in elitneJedinke:
            novaGen.append(e)

        for _ in range(brojPopulacije-2):
            
            parent1, parent2 = random.sample(parents, 2)
            child = ukrstanje(parent1, parent2)
            child = mutacija(child,brojCvoreva)
            novaGen.append(child)

        populacija = novaGen
        
        if (time.time() - startTime)>vrijemeTrajanja:
            break
        
    print("Populacija na kraju",populacija)
    return min(populacija, key=lambda s: selekcija(s, graf))


In [23]:
#Grafovi

g1 = {
    1: [2, 3],
    2: [1,3,4],
    3: [1,2,4],
    4: [2,3,5,6],
    5: [4,6,7,8,9],
    6: [4,5],
    7: [5],
    8: [5],
    9: [5]
}

g2 = {
    1: [2, 3],
    2: [1,4,3],
    3: [1,2],
    4: [2]
}

g3 = {
    1: [2,3],
    2: [1,3,4,5],
    3: [1,2,20],
    4: [2,5],
    5: [2,4,6,7],
    6: [5,12],
    7: [5,8,9],
    8: [7,9],
    9: [7,8,10,14,15,16],
    10: [9,11,13,14,16],
    11: [10,12,13,14],
    12: [6,11],
    13: [10,11,14],
    14: [9,10,11,13,15,16],
    15: [9,10,14,16],
    16: [9,10,14,15,17],
    17: [16,18,19,20],
    18: [17,19,20],
    19: [17,18,20],
    20: [3,17,18,19]
}

g4 = {
    1: [2,3,4],
    2: [1,5,6],
    3: [1,7,8],
    4: [1,9,10],
    5: [2,10,11],
    6: [2,7,12],
    7: [3,6,13],
    8: [3,9,14],
    9: [4,8,15],
    10: [4,5,16],
    11: [5,12,17],
    12: [6,11,18],
    13: [7,14,18],
    14: [8,13,19],
    15: [9,16,19],
    16: [10,15,17],
    17: [11,16,20],
    18: [12,13,20],
    19: [14,15,20],
    20: [17,18,19]
}

g5 = {
    1: [4],
    2: [4],
    3: [4],
    4: [1,2,3,5], 
    5: [4,11], #5:[4,6]
    6: [7,11], #6:[5,7]
    7: [6,8,9,10],
    8: [7],
    9: [7],
    10: [7],
    11: [5,6] #
}

g6 = {
    1: [4],
    2: [4],
    3: [4],
    4: [1,2,3,5], 
    5: [4,6],
    6: [5,7],
    7: [6,8,9,10],
    8: [7],
    9: [7],
    10: [7],
}

In [24]:
# Pokretanje genetskog algoritma
startTime = time.time()
print(napraviGraf("graf125-9.txt"))
gen =genetskiAlgoritam(napraviGraf("graf8-2-4.txt"),20)
print("Minimalni slabo povezani dominirajuci skup:", gen, len(gen))
print("--- %s sekundi ---" % (time.time() - startTime))

{'1': ['2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '38', '39', '40', '41', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '54', '55', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '88', '89', '90', '91', '92', '93', '94', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '107', '110', '111', '112', '113', '114', '115', '116', '117', '119', '120', '121', '122', '123', '124', '125'], '2': ['1', '4', '5', '6', '7', '8', '9', '10', '11', '12', '14', '16', '17', '18', '19', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '54', '57', '58', '59', '